In [2]:
import os
import pydicom
import glob
import pandas as pd
from datetime import datetime
import gzip
import io
import shutil
from pyorthanc import Orthanc

In [3]:
def get_orthanc_client():
    # Initialize orthanc client
    orthanc = Orthanc('http://localhost:8042')
    orthanc.setup_credentials('dev-user-alta', 'SyTP&8JbKFx@a6R65^sE`Z$') 
    return orthanc

orthanc_client = get_orthanc_client()

In [4]:
# load segmented cases  
prostate_lesion_seg = pd.read_csv("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/train/lesion_dataset_1909_20240131_OB.csv", sep=";")

# load cases proposed by Lucas
cases_to_annotate = pd.read_excel("meta_data_lesion/20240702_Lucas_Cases_mit_KI_bewerten.xlsx")

# check for overlap
prostate_lesion_seg[prostate_lesion_seg["study_orthanc_id"].isin(cases_to_annotate["study_orthanc_id"])]

,study_orthanc_id,class,region,zone,segment_id,count,study_date,patient_id,csPCa,case_class,healthy,StudyInstanceUID,class_id


In [5]:
seq_map = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/sequence_mapping/sequence_mapping_13681_studies_20240807.csv", sep=";") 
seq_map.head(3)

,t2w_tra_id,t2w_tra_sd,t2w_sag_id,t2w_sag_sd,t2w_cor_id,t2w_cor_sd,adc_tra_id,adc_tra_sd,adc_sag_id,adc_sag_sd,...,dwi_tra_id,dwi_tra_sd,dwi_sag_id,dwi_sag_sd,dwi_cor_id,dwi_cor_sd,Manufacturer,study_orthanc_id,PatientID,StudyDate
0,7f6c6664-416331da-175b9083-1a67fdfa-064bf673,t2_tse_tra,7cd0bca9-73752b20-fb4f318f-bf79dc39-95166304,t2_tse_sag_p2,10f846e9-f30ceaf6-826e5959-1af9b396-9dd682cc,t2_tse_cor_p2_320,232a75a0-af68dbfe-cbc0afec-f9aa3347-bd6541f7,ep2d_diff_b50_500_1000_tra_adc_dfc,NaN,NaN,...,1ac49251-c9662c57-d70bf60b-0e1db131-19ad825c,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,fcac7d02-2ef30e51-0572b869-91e8f18b-938eb178,ep2d_diff_b50_500_1000_cor_tracew_dfc,siemens,786714b5-a6309313-214532e5-e92d1dff-308695f6,26586,20220201.0
1,b568bc9b-6a9d9e07-65793909-7a453d4c-784a5238,t2_tse_tra,d505ddb2-ea663427-5d079838-795bfb31-4d1f6a02,t2_tse_sag_p2,c036a794-9e549195-c6a59f77-ba67c619-2a70bba1,t2_tse_cor_p2_320,84669307-e90afc45-d3d6f7d0-fef92916-36c603bf,ep2d_diff_b50_500_1000_tra_adc_dfc,NaN,NaN,...,d7dfb776-f693fbfd-62795a90-62e28fda-d1a86544,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,NaN,NaN,siemens,e27d17d9-47c79ade-2218d504-e26738cd-568156b3,31906,20220201.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,siemens,21915507-0546865f-dfde6bc5-77369bc3-bb431e29,31906,20220201.0


In [6]:
# filter non-existing cases out
cases_to_annotate = cases_to_annotate[cases_to_annotate["study_orthanc_id"].isin(seq_map["study_orthanc_id"])]

dataset_annotate = seq_map[seq_map["study_orthanc_id"].isin(cases_to_annotate["study_orthanc_id"])]
dataset_annotate_full = dataset_annotate[~(dataset_annotate['t2w_tra_id'].isna() | dataset_annotate['adc_tra_id'].isna() | dataset_annotate['dwi_tra_id'].isna())]

In [7]:
dataset_annotate_full

,t2w_tra_id,t2w_tra_sd,t2w_sag_id,t2w_sag_sd,t2w_cor_id,t2w_cor_sd,adc_tra_id,adc_tra_sd,adc_sag_id,adc_sag_sd,...,dwi_tra_id,dwi_tra_sd,dwi_sag_id,dwi_sag_sd,dwi_cor_id,dwi_cor_sd,Manufacturer,study_orthanc_id,PatientID,StudyDate
5,850f8c78-d1cca6b3-bccb39af-70313b15-effb54f2,t2_tse_tra,2e2d0d6d-9e23e9b3-8be22e35-b479d897-08a398a2,t2_tse_sag_p2,NaN,NaN,f164761a-fd601e2a-2bb8a74c-14ddcd47-a7ef7c71,ep2d_diff_b50_500_1000_tra_adc_dfc,NaN,NaN,...,45c3b3d5-c9d283d5-0e15f547-896a81dc-6a8c8306,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,NaN,NaN,siemens,8f4a4be1-931471c9-30efc293-ed3d79b0-29f0f99b,32382,20220201.0
6,cc704590-cd0f05c9-d2d492cf-d4f79858-7dc9e35a,t2_tse_tra,3fb875fc-147bf84d-8ddd5fce-1a3036ec-6328055d,t2_tse_sag_p2,673bbec0-1796c80f-1d6b0392-20e1ebe6-ee51f2ea,t2_tse_cor_p2_320,cf084739-92fc9d91-81939738-88466c7f-9a2ce9d2,ep2d_diff_b50_500_1000_tra_adc_dfc_mix,NaN,NaN,...,ce3ea492-24e31da6-614dc702-e04ba3cd-ab5bfca0,ep2d_diff_b50_500_1000_tra_tracew_dfc_mix,NaN,NaN,NaN,NaN,siemens,bc2ca49c-f1432059-fd36e8cc-abd83766-5532070a,22520,20220201.0
8,417f1752-5de8edfe-934aad6f-ab586715-2f346a9c,t2_tse_tra,df5a22e7-b34af7af-9161f759-df42a768-b6db850c,t2_tse_sag_p2,e39ebbaf-9cf93e07-e5df2d23-8772de82-4fb0cff3,t2_tse_cor_p2_320,9ac4f05c-9093fc98-8c440657-d1e15a00-d6460415,ep2d_diff_b50_500_1000_tra_adc_dfc_mix,NaN,NaN,...,f5dab731-58844539-e95c8892-43a1ea87-59cdef83,ep2d_diff_tra_ 50_500_1000_5mm_becken_tracew_d...,NaN,NaN,6c26502e-f824f19e-fa734037-02de6872-4d7db693,ep2d_diff_b50_500_1000_cor_tracew_dfc,siemens,ddd04df3-f36da29e-a7d01715-0612285d-d3551bb7,32395,20220203.0
26,f10dd8e8-8a642e69-0372feac-636adf31-6e6c7b7e,t2_tse_tra,5c34664b-22ea60f5-db7ec448-3cab61a3-5e128c98,t2_tse_sag_p2,NaN,NaN,df48ee16-eed8527f-6d242279-e97d3efb-fc303d0f,ep2d_diff_b50_500_1000_tra_adc_dfc,NaN,NaN,...,cb72c670-9f8c2e56-c1b7739b-b8893f58-5eef5f04,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,NaN,NaN,siemens,d6440358-34d25cfe-9c2b8886-d2f94459-9f64a0f4,24705,20220208.0
30,53508cf0-f0afd946-0021c997-b718068e-586fbe8f,t2_tse_tra,6fbcae73-27e5d2fd-aab7ac67-a9d22fbb-20411df2,t2_tse_sag_p2,dae21752-a254d394-af45ad95-891fe76a-ce11f813,t2_tse_cor_p2_320,f524dc4f-847dfc6b-fd382c10-3e6b811c-609aeddf,ep2d_diff_b50_500_1000_tra_adc_dfc_mix,NaN,NaN,...,555ba015-34ec0bf5-85f477f1-a800caf1-fd6bb291,ep2d_diff_b50_500_1000_tra_tracew_dfc_mix,NaN,NaN,NaN,NaN,siemens,e920547b-5fe23dbd-f640d511-39dbaab0-6e6766a9,aaa284264611,20220209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13022,eae78e5e-2ddd492d-a0351877-5b5759fc-24417237,t2_tse_tra,66c0dbe6-748197ac-05c9e554-de343560-a4776b16,t2_tse_sag_p2,NaN,NaN,5f459eef-ef4ce60b-b16298f2-76715d3d-60fafb04,ep2d_diff_b50_500_1000_tra_adc,NaN,NaN,...,65a3a06b-af34c81d-3fa3a423-8a3ac7e4-d9007af7,ep2d_diff_b50_500_1000_tra_tracew,NaN,NaN,c1438460-b963e376-3600696d-8d10729f-4e461c08,ep2d_diff_b50_500_1000_cor_tracew,siemens healthineers,82e0eaaa-8094ebbd-94d85621-fd39d177-3d7317d6,15220,20230619.0
13023,dc692f18-7fc464d4-9d6c86b5-c993b64a-003039d7,t2_tse_tra,358a0d42-8b80806b-f77200e0-8872e77a-11f35878,t2_tse_sag_p2,NaN,NaN,096812df-0ab9ca84-3224de68-efb54cb9-bc4b2f4b,ep2d_diff_b50_500_1000_tra_adc,NaN,NaN,...,00e63327-7bcd16c8-2a49fca4-ce0b110e-f487c5c5,ep2d_diff_b50_500_1000_tra_tracew,NaN,NaN,de4e1204-74cee8fa-a585435f-b0458147-94dc9b0d,ep2d_diff_b50_500_1000_cor_tracew,siemens healthineers,713ce0e3-4f59b2fb-761d16dd-31eb7801-67df0181,41945,20230619.0
13024,a68af768-2c0f353e-e82aeb5e-e538ec0f-3df4bbaa,t2_tse_tra 4mm,bb6610bd-d1fcfab2-732062ee-56d90bd8-2f49a582,t2_tse_sag_3mm,1197317b-e0788182-2e5b36e0-f625ac50-cefdc076,t2_tse_cor_320_p2_4mm,39ab0592-be7c634d-6c7ee017-87535015-9287fbf1,ep2d_diff_tra_b50-500-1000_4mm_adc_dfc_mix,NaN,NaN,...,17f19ec4-f062146e-6d2050fd-82f18d90-fca5678b,ep2d_diff_tra_b50-500-1000_4mm_tracew_dfc_mix,NaN,NaN,NaN,NaN,siemens,be94e060-033e15a7-f15f4ad1-f7ec9589-f86bcd33,AAA171024669,20180125.0
13034,6a05dbc0-581ad3b6-6d79828b-a269890e-51c92314,t2_tse_tra

In [8]:
# remove recently segmented cases from the list
lesion_studies_processed_processed = glob.glob("/data/oleksii/alta-ai.com/alta-ai-orthanc-backup/2024-06-30/prostate_lesion/ProcessingState.PROCESSED/*/seg.dcm.gz")
lesion_studies_processed_unsure = glob.glob("/data/oleksii/alta-ai.com/alta-ai-orthanc-backup/2024-06-30/prostate_lesion/ProcessingState.UNSURE/*/seg.dcm.gz")


def unzip_gz_file(input_file_path, output_file_path):
    with gzip.open(input_file_path, 'rb') as f_in:
        with open(output_file_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    
meta_info = []
for seg in lesion_studies_processed_processed+lesion_studies_processed_unsure:
    orthancID = seg.split("/")[-2]
    
    if seg.endswith(".gz"):
        seg_target = seg.replace(".gz", "")
        unzip_gz_file(seg, seg_target)
    else:
        seg_target = seg        
    
    dataset = pydicom.dcmread(seg_target)
    StudyInstanceUID = dataset[(0x0020, 0x000d)].value
    study_orthanc_id = orthanc_client.post_tools_lookup(data=StudyInstanceUID)
    if study_orthanc_id:
        study_orthanc_id = study_orthanc_id[0]['ID']
    else:
        study_orthanc_id = None
    meta_info.append({"series_orthanc_id": orthancID, "StudyInstanceUID": StudyInstanceUID, "study_orthanc_id": study_orthanc_id})
avaliable_lesions = pd.DataFrame(meta_info)

print(len(dataset_annotate_full))
dataset_annotate_full_filtered = dataset_annotate_full[~dataset_annotate_full["study_orthanc_id"].isin(avaliable_lesions['study_orthanc_id'])]
print(len(dataset_annotate_full))

2522
2522


In [9]:
# sent to Lucas for double checking
# dataset_annotate_incomplete_seq = dataset_annotate[~dataset_annotate["study_orthanc_id"].isin(dataset_annotate_full["study_orthanc_id"])]
# dataset_annotate_incomplete_seq.to_csv("annotate_incomplete_seq_map.csv", sep=";")


In [10]:
class_list = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/datasets/classification/klassenliste_matched_with_pacs_20240411.csv", sep=";")
current_anno_batch = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/datasets/lesion/annotate_batch_01_701_seq_map.csv", sep=";")
dataset_annotate_class = dataset_annotate_full_filtered[~dataset_annotate_full_filtered["study_orthanc_id"].isin(class_list["study_orthanc_id"])]
# dataset_annotate_class.to_csv("annotate_batch_01_701_seq_map.csv", sep=";")

In [11]:
dataset_annotate_ = dataset_annotate_full_filtered[~dataset_annotate_full_filtered["study_orthanc_id"].isin(current_anno_batch["study_orthanc_id"])]

dataset_annotate_.to_csv("annotate_batch_02_1563_seq_map.csv", sep=";")

In [15]:
sequence_selected_path = "/home/oleksii/projects/ohif-orthanc-postgres-docker/datasets/lesion/meta_data_lesion/Batch 2 alta AI.xlsx"
# sequence_selected = pd.read_csv(sequence_selected_path, sep=';')
sequence_selected = pd.read_excel(sequence_selected_path)
sequence_selected



,t2w_tra_id,t2w_tra_sd,t2w_sag_id,t2w_sag_sd,t2w_cor_id,t2w_cor_sd,adc_tra_id,adc_tra_sd,adc_sag_id,adc_sag_sd,...,dwi_tra_sd,dwi_sag_id,dwi_sag_sd,dwi_cor_id,dwi_cor_sd,Manufacturer,study_orthanc_id,PatientID,StudyDate,Status
0,690cd0ee-d6c35994-ed36ead0-f209c08b-d8711fa2,t2_tse_tra,2f87d426-5323e999-abe34135-e6a6919e-800bc96e,t2_tse_sag_p2,NaN,NaN,7020edcf-f3a2c2b7-1371e243-909ba945-76e2f14e,ep2d_diff_b50_500_1000_tra_adc_dfc,NaN,NaN,...,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,eab6957c-e343048a-394554fb-d1b4eb41-548fac9f,ep2d_diff_b50_500_1000_cor_tracew_dfc,siemens,e358ee62-5680b1ee-57ae8989-9dd60caf-5b4a1722,27686,20220919,fehlt noch
1,d20f1ca6-3f2c2eb1-b1a5c095-00f45e97-577460df,t2_tse_tra_320_p2,8ab4267c-60f4f102-039c2272-f29c6ab2-d179df83,t2_tse_sag,NaN,NaN,b8978fa2-b856c715-e19d55a4-d94b059b-be5d0e0e,ep2d_diff_tra_b50-500-1000_3mm_adc,NaN,NaN,...,ep2d_diff_tra_5mm _becken_tracew,NaN,NaN,"af1512c6-d0f98d90-3c5e2182-c7686863-aff07d73,5...","ep2d_diff_cor_b50-500-1000_3mm_tracew,ep2d_dif...",siemens,09016017-2bf725f3-59767286-9649e78a-2c7b3528,aaa302839871,20220922,fehlt noch
2,63b85894-108fa526-7cd9ccb6-8e17dd52-64216dd6,t2_tse_tra,7404c4a1-406a27d3-e77a3918-edce07e4-a43dcc31,t2_tse_sag_p2,NaN,NaN,a8d5efbb-517106b7-f56269ce-8b4481e1-0d7d5d2f,ep2d_diff_b50_500_1000_tra_adc_dfc,NaN,NaN,...,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,728f77f1-d128fcbc-0f1f01b6-5a90a1d5-efb4250d,ep2d_diff_b50_500_1000_cor_tracew_dfc,siemens,32445f31-0b995abc-0870bfd0-a9d674d9-71df32da,26473,20220928,fehlt noch
3,622e9d49-3b3aa3a1-acb88eba-e400922f-71a3de65,t2_tse_tra,5787ffac-acad72fd-78abd167-fc78bc20-0973db18,t2_tse_sag_p2,NaN,NaN,b035b9f6-08a38cc5-edb41447-19e2de3f-8e5622f6,ep2d_diff_b50_500_1000_tra_adc_dfc,NaN,NaN,...,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,2a47de58-ed2b564e-e8db6d3c-64bb34e2-3958c562,ep2d_diff_b50_500_1000_cor_tracew_dfc,siemens,0f583878-8f6be4d1-3bdbcd5f-f2ab0bf3-6c7c7ecf,20727,20220630,fehlt noch
4,201cdd73-790f33c5-c1cdc807-bb50eadc-e5fe6cae,t2_tse_tra,c3979cd9-50e5570e-27035e1a-952f1455-f01ed606,t2_tse_sag_p2,7198eb2c-651a4793-42ee6fb1-af4a96f8-70344ec6,t2_tse_cor_p2_320,05024ed3-ad9a49ea-98994445-af47e285-620b401d,ep2d_diff_b50_500_1000_tra_adc_dfc_mix,b1afa1ef-d630f087-fd15f55a-48bebf95-c7427d2a,ep2d_diff_b50_500_1000_sag_adc_dfc,...,ep2d_diff_b50_500_1000_tra_tracew_dfc_mix,0590701a-96c418fe-9a2234aa-d6ea7a53-bfdd83a2,ep2d_diff_b50_500_1000_sag_tracew_dfc,2343bc1c-9e3cd446-475aceeb-219d468d-38d5db67,ep2d_diff_b50_500_1000_cor_tracew_dfc_mix,unknown,337067b1-22df2da7-559672b3-6410b9b4-65d3937b,33558,20220623,fehlt noch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,d0c2acc8-e74410a8-0bf13658-5a938dd2-bef05bfe,t2-tse-tra,d180678d-5a1ff191-7157b5e1-0feac36c-71a54aba,t2-tse-sag,62ea889f-d66d7fd2-c99f6516-8a8b4009-23ffd66a,t2-tse-stir-cor,75ba4eaf-545d6b89-8a752584-cf6762f6-7d8e7f3c,ep2d_diff_zoomit_b50_800_1500_tra_2mm_adc,NaN,NaN,...,"ep2d_diff_zoomit_b50_800_1500_tra_2mm_tracew,e...",NaN,NaN,NaN,NaN,siemens healthineers,127e90d0-6ae0bd7a-93215adf-2f87a347-186c1007,W6824,20240524,fehlt noch
2336,a487152b-a561aae6-8985e635-e8860a29-c3c126da,t2_tse_tra_p2,33d3b064-89ab1b07-ae4f7d61-478b87f5-2c69ab25,t2_tse_sag_p2,82580059-c43d6f20-cbd498fa-7a331967-e7ee6518,t2_tse_cor_p2,00a454ca-356e3baf-9c107707-eb3ee98d-0d28b0dd,ep2d_d_diff_50_400_1000_tra_adc,NaN,NaN,...,"ep2d_d_diff_50_400_1000_tra,ep2d_d_diff_50_400...",NaN,NaN,NaN,NaN,siemens healthineers,94dc79c0-a84f77bf-3882c641-a8ab01dc-c9d323a4,127936,20240521,fehlt noch
2337,57af534c-f9a9b76e-4fccb62f-df242115-db5b22bc,t2_tse_tra,42c2e06b-5a3d1af5-372d1506-06f5a44f-c05e8e0f,t2_tse_sag_p2,NaN,NaN,07cbde9d-0c10852a-91676920-5ea9886c-09f04f53,ep2d_diff_b50_500_1000_tra_adc,NaN,NaN,...,ep2d_diff_b50_500_1000_tra_tracew,NaN,NaN,ba4b258f-303ca790-55e53f8c-abc7cccc-fe0d8669,ep2d_diff_zoomit_b100_900_cor_tracew,siemens healthineers,dc7c7681-ec4b3c58-29976fc0-dfb381c8-7f5811b5,44881,20240605,fehlt noch
2338,9a6857d9-6311

In [24]:
dataset_annotate_class = dataset_annotate_[~dataset_annotate_["study_orthanc_id"].isin(class_list["study_orthanc_id"])]
dataset_annotate_class

,t2w_tra_id,t2w_tra_sd,t2w_sag_id,t2w_sag_sd,t2w_cor_id,t2w_cor_sd,adc_tra_id,adc_tra_sd,adc_sag_id,adc_sag_sd,...,dwi_tra_id,dwi_tra_sd,dwi_sag_id,dwi_sag_sd,dwi_cor_id,dwi_cor_sd,Manufacturer,study_orthanc_id,PatientID,StudyDate
671,cae11820-7923627d-19214eab-5b4b89ec-6a310c9e,t2tse_tralr,5d6015fd-e339d48f-a83d68ae-bbac0d6c-2d71565a,t2tse_sag,5b0c578b-de6d08f0-2ca57466-e4bcdc1e-b7375cf4,t2tse_corlr,49e1f1ad-ed959808-e587f969-31c28272-27d31e9c,dadc,NaN,NaN,...,"0c509f00-a40fa983-4c1668e4-c626f9ce-446f62eb,0...","dwi_4b (0.100.400.1000),sdwi_b0,sdwi_b100,sdwi...",NaN,NaN,NaN,NaN,philips,aa3be8e4-c6468dd4-b7351d93-59e54255-3fe7d946,Q69623,20220112.0
725,c3d77fe6-b7dfb5fc-f0fdf28c-7adc4187-bac14e81,t2_tse_3mm_tra,df5f26e3-dd5ac2d1-87013124-5705fb1c-c6863f5d,sag t2 tse fh,686aca50-e2628ef0-697a12e0-d83c6e5a-928a90a8,t2_tse_3mm_cor,9b27f596-d4c233de-6f0a2c9a-f5a9259e-06e96100,dadc dwi_b100_400_800,NaN,NaN,...,"0b953dca-02b70962-5056986a-55f8597e-658a5421,1...","sdwi_b100,sdwi_b800,dadc dwi_b100_400_800,dwi_...",NaN,NaN,NaN,NaN,philips,b97541c2-e9d4c3f7-e408b37f-2cb3bca2-9ed8fe96,1223754,20220208.0
8684,cedbeb88-351ab00f-03e33822-f5f19696-d50a81d6,t2-tse-zoomit-epi-diff-tra_tracew,ae97737d-dea87075-f6e1e87d-4ddaa880-fb9e5341,t2-tse-blade-sag,ff656a1a-f4474033-2ecf282c-4581c4f5-d618777d,t2-tse-cor,da26b5ec-db7e572b-9fa07019-951bf6cc-678546a9,t2-tse-zoomit-epi-diff-tra_adc,NaN,NaN,...,cedbeb88-351ab00f-03e33822-f5f19696-d50a81d6,t2-tse-zoomit-epi-diff-tra_tracew,NaN,NaN,NaN,NaN,siemens,18acbf23-94ace654-2b6083e6-4940ed33-b52ea72d,0040328881,20191113.0
8987,57514a99-57249fb9-93eee8ea-adc7a679-ed148d60,t2 tse tra 3mm,f4cd067a-c3d3e6da-c8d92157-97c5b992-1e0ed285,t2w_tse_sag,b284dfd2-290bdcf7-9c7dc429-564d3f2b-b2d093b8,t2 tse cor 3mm,69c94cdb-8e33de5a-b2868e26-960bb3b8-b11fa4f5,dadc map,NaN,NaN,...,9cc104d4-3e122abc-935574e4-1c3632a5-6148d2ec,dwi_3b,NaN,NaN,NaN,NaN,philips,53a3b9e5-f197c7f8-adba1b01-fa911ef8-f1f2319a,5725,20190918.0
11287,cdfbb6ff-da5441aa-6e3e0c17-b12b7010-fe7aa049,cs t2w_tse_ax kb,4efbfc14-4c2d506e-81f54986-9139038b-04d52326,t2w_tse_sag kb,28a4a83e-324cce8d-4df29a04-3b2d246a-a0e5c5ec,t2w_tse_cor kb,8b59c3a4-b0119e72-32ffaf5e-98377427-32815fc3,dadc prostata dwi_0_800 1500 sense,NaN,NaN,...,"24df11c1-cec89587-23d76904-f0d34c77-38cd2e27,1...","dwi_0_ 50 800 1500 sense,sdwi b1500 sense,dwib...",NaN,NaN,"fb6fdf82-a03edb99-f393bdd5-6ea5c7e6-f63d9bb6,9...","slab mip dwibs,sinvert mip dwibs,sinvert slab ...",philips,ef1ba8fb-1cd063b7-374a5ac8-c88f217e-40c0e251,151688,20221118.0
11310,f0743dff-b81a5ead-23e0df42-9a22559b-cddbb866,t2-tse-tra,ae32b77e-c56f8b04-5b31df34-bbcb0105-01be8d6c,t2-tse-blade-sag,9668c6a1-14d56bd3-da9fa680-c3055b26-08721490,t2-tse-stir-cor,961a23b6-b871b186-39bc5c38-a8ee3182-41f999be,t2-tse-epi-tra_adc,NaN,NaN,...,51f10032-e41bdcf1-9a5cf5c5-49b2a87e-f33fe711,t2-tse-epi-tra_tracew,NaN,NaN,NaN,NaN,siemens healthineers,9f71dfa5-500def80-5eaf8acd-3f385f3c-310a7057,GAP752188,20221118.0
11367,62c3109a-02bb4972-79dd3be6-3fa060dc-334d1280,t2w_tse_ax,NaN,NaN,NaN,NaN,ceef7f92-745b196d-21de0741-687988f8-18476eac,dadc,NaN,NaN,...,"b66f017b-87c35249-8e79d182-96a094c2-d43ab4a9,7...","dwi_0/50/1000/1500,reg - dwi_0/50/1000/1500b 3...",NaN,NaN,NaN,NaN,philips,8e7f973e-2d2d274b-41d06a9c-95c9a7ff-7c64c6ff,3152059604,20221111.0
11383,e19521fe-15f9d8ef-e256cc0d-f39151e3-d44d5ce4,t2 tse tra 0.4x0.7x3.0,9f1fe936-987740d8-d6e56def-d19b42e4-788edbd2,sag t2 tse,2d7135c9-577110ad-b1790bd8-c1499b60-2c3a0f49,t2 tse cor,ff3f98a0-7b6daea5-898fd4b8-1588f248-bfa62526,dadc map,NaN,NaN,...,"b35d9338-493fab9f-d165b455-b88a8345-034b99fc,7...","dwi_b50_500_1000_2x2mm,sswip dwi_b50i,swip dwi...",NaN,NaN,NaN,NaN,philips,aaeb4dcd-2d910d85-1036c57d-d4db81af-577e6887,3590,20221130.0
11730,eb08becd-3540f53c-f05d8224-7d24c276-52b858da,t2 tse tra 3mm,a8fa8386-ef301845-8cfde2b6-747e8401-e5e94a9f,t2 tse sag 3mm,163fb0ac-833c057d-0c0eac19-79a93b58-4aa625fd,t2 tse cor 3mm,600be566-8e667180-5daf72fb-bf40d4dc-01a419a3,dadc,NaN,NaN,...,"b20929cd-9715e9d5-2de

In [ ]:
# /data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-batch-anno-Segmentation-AI/0ab2738c-b90a5580-8e2f52c3-9343062c-f2e46b91/201c3b3ecccc09a4b0942af2d32dde1eb9f54f75_AI_softmax_0_lesion.nrrd
# /data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-batch-anno-Segmentation-AI/0ab2738c-b90a5580-8e2f52c3-9343062c-f2e46b91/201c3b3ecccc09a4b0942af2d32dde1eb9f54f75SegmentationAI_lesion.seg.nrrd
# /data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-batch-anno-Segmentation-AI/0ab2738c-b90a5580-8e2f52c3-9343062c-f2e46b91/201c3b3ecccc09a4b0942af2d32dde1eb9f54f75SegmentationAI_zone.seg.nrrd
